In [ ]:
#1.Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#2.원본 json데이터 불러오기
import json

input_path = "/content/drive/MyDrive/KoGPT2_data/민원(콜센터) 질의응답_K쇼핑_결제_Training.json"

with open(input_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

print(f"총 샘플 수: {len(raw_data)}")

총 샘플 수: 213933


In [ ]:
#3.KoGPT2 학습 포맷으로 전처리
formatted_data = []

for i in range(len(raw_data) - 1):
    q = raw_data[i]
    a = raw_data[i + 1]

    if (
        q["화자"] == "고객"
        and a["화자"] == "상담사"
        and q.get("고객질문(요청)", "").strip()
        and a.get("상담사답변", "").strip()
    ):
        user_q = q["고객질문(요청)"].strip()
        agent_a = a["상담사답변"].strip()

        # 너무 짧은 무의미 응답 제거
        if len(agent_a) < 8:
            continue

        formatted_data.append({
            "text": f"사용자: {user_q}\n답변: {agent_a}"
        })

print(f"전처리된 샘플 수: {len(formatted_data)}")

전처리된 샘플 수: 5054


In [ ]:
#4.파일로 저장
output_path = "/content/drive/MyDrive/KoGPT2_data_cleaned/Cleaned.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=2)

print("✅ 저장 완료:", output_path)

✅ 저장 완료: /content/drive/MyDrive/KoGPT2_data_cleaned/Cleaned.json


In [ ]:
#5.확인(샘플 출력)
for sample in formatted_data[:5]:
    print(sample["text"])
    print("—" * 30)

사용자: 삼성 카드는요?
답변: 아 네 삼성 카드 가능합니다.
——————————————————————————————
사용자: 그럼 결제 금액이 얼마죠?
답변: 네 십사만 칠천원 입니다.
——————————————————————————————
사용자: 그럼 할부로 하면요?
답변: 고객님 십오만 구천원입니다.
——————————————————————————————
사용자: 그러면 주문한 상품이랑 카드 정보만 알면 된다는거죠?
답변: 네 고객님 맞습니다.
——————————————————————————————
사용자: 로션 바르고 다음에 바르는거예요?
답변: 네 마무리 단계에서 해주시면 될 것 같습니다.
——————————————————————————————


In [ ]:
#파인튜닝코드
#1.모델/토크나이저 로딩
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"  # 공개된 KoGPT2 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # padding 안정화
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

Embedding(51201, 768)

In [ ]:
#2.데이터 불러오기 및 전처리
import json
from datasets import Dataset

with open("/content/drive/MyDrive/KoGPT2/KoGPT2_data_cleaned/민원(콜센터) 질의응답_K쇼핑_결제_Cleaned.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

dataset = Dataset.from_list(raw)

def tokenize(example):
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors=None  # Trainer와 호환되게 tensor로 반환하지 않음
    )
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding


tokenized_dataset = dataset.map(tokenize, remove_columns=["text"])

Map:   0%|          | 0/5054 [00:00<?, ? examples/s]

In [ ]:
#3.학습설정
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",  #epoch마다 저장
    report_to="none",
    fp16=True,  # GPU 있으면 True 가능
    remove_unused_columns=False
)

In [ ]:
#4.Trainer 구성 & 학습시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)
#체크포인트 불러오기
#checkpoint_path = "/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2/checkpoint-3792"
#trainer.train(resume_from_checkpoint=checkpoint_path)

#처음 학습
trainer.train()

<ipython-input-22-e30dc65ca93e>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


AttributeError: 'NoneType' object has no attribute 'load_state_dict'

In [ ]:
#5.모델저장
model.save_pretrained("/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final")
tokenizer.save_pretrained("/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final")

('/content/drive/MyDrive/KoGPT2_finetuned2_final/tokenizer_config.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final/special_tokens_map.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final/vocab.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final/merges.txt',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final/added_tokens.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final/tokenizer.json')

In [ ]:
#6.테스트
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

def generate_answer(user_input, max_length=80):
    prompt = f"사용자: {user_input}\n답변:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("답변:")[-1].strip()

questions = [
    "배송은 얼마나 걸리나요?",
    "환불 가능한가요?",
    "결제 수단 바꾸고 싶어요",
    "카드 등록하면 할인받을 수 있나요?",
    "일시불로 하면 더 싼가요?"
]

for q in questions:
    answer = generate_answer(q)
    print(f"질문: {q}")
    print(f"응답: {answer}")
    print("-" * 40)

질문: 배송은 얼마나 걸리나요?
응답: 배송이 어려우면 다음 주 화요일 수령 가능합니다.
----------------------------------------
질문: 환불 가능한가요?
응답: 네 고객님 확인해 보겠습니다.
----------------------------------------
질문: 결제 수단 바꾸고 싶어요
응답: 아 네. 맞습니다. 고객님.
----------------------------------------
질문: 카드 등록하면 할인받을 수 있나요?
응답: 네, 고객님 할인 받을 수 있습니다.
----------------------------------------
질문: 일시불로 하면 더 싼가요?
응답: 일시불하시면 만원 할인되서 오만 팔천구백원입니다.
----------------------------------------


In [ ]:
#추가 데이터 학습
#1.기존 모델 불러오기
from transformers import AutoTokenizer, AutoModelForCausalLM

base_path = "/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final"  # 너가 이전에 저장한 경로
tokenizer = AutoTokenizer.from_pretrained(base_path)
model = AutoModelForCausalLM.from_pretrained(base_path)

tokenizer.pad_token = tokenizer.eos_token  # 안전 설정

In [ ]:
#2.새 데이터 로드 및 전처리
import json
from datasets import Dataset

#추가할 데이터 로드
with open("/content/drive/MyDrive/KoGPT2/KoGPT2_data_QA/결제QA_다양화.json", "r", encoding="utf-8") as f:
    new_data = json.load(f)

# Huggingface Dataset 포맷으로 변환
dataset = Dataset.from_list(new_data)

def tokenize(example):
    text = example["text"]  # 반드시 문자열
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    encoding["labels"] = encoding["input_ids"]  # 딥카피 말고 그냥 복사
    return encoding

tokenized_dataset = dataset.map(tokenize, remove_columns=["text"])

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
#3.학습설정 및 학습시작
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final_v2",
    per_device_train_batch_size=4,
    num_train_epochs=1,  # 추가 1 epoch만 진행
    logging_dir="./logs",
    remove_unused_columns=False,
    save_strategy="no",  # 중간 checkpoint 저장 없이
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-20-7e60b29c1df5>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=8, training_loss=0.15089300274848938, metrics={'train_runtime': 5.3801, 'train_samples_per_second': 5.576, 'train_steps_per_second': 1.487, 'total_flos': 7838760960000.0, 'train_loss': 0.15089300274848938, 'epoch': 1.0})

In [ ]:
#4.모델저장
trainer.save_model("/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final_v2")
tokenizer.save_pretrained("/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final_v2")

('/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/tokenizer_config.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/special_tokens_map.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/vocab.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/merges.txt',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/added_tokens.json',
 '/content/drive/MyDrive/KoGPT2_finetuned2_final_v2/tokenizer.json')

In [ ]:
#5.테스트
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/KoGPT2/KoGPT2_finetuned2_final_v2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

# 응답 생성 함수
def generate_multiple_answers(question, num_return_sequences=3):
    prompt = f"사용자: {question}\n답변:"
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_length=60,
        do_sample=True,
        top_k=50,
        top_p=0.85,
        temperature=0.9,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=4,
        repetition_penalty=1.5,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    answers = []
    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        answer = decoded.split("답변:")[-1].strip()
        answers.append(answer)
    return answers

def select_best_response():
    while True:
        user_question = input("질문을 입력하세요 (종료하려면 'exit'): ")
        if user_question.strip().lower() == "exit":
            break

        answers = generate_multiple_answers(user_question, num_return_sequences=3)

        print("\n생성된 응답들:")
        for i, ans in enumerate(answers, start=1):
            print(f"{i}. {ans}")

        while True:
            try:
                choice = int(input("\n✅ 맘에 드는 응답 번호를 선택하세요 (1~3): "))
                if 1 <= choice <= 3:
                    print(f"\n최종 선택된 응답: {answers[choice - 1]}\n")
                    break
                else:
                    print("⚠️ 1~3 사이의 숫자를 입력해주세요.")
            except ValueError:
                print("⚠️ 숫자로 입력해주세요.")


select_best_response()

📥 질문을 입력하세요 (종료하려면 'exit'): 카카오페이로 결제 가능한가요?

🔍 생성된 응답들:
1. 네, 주문 시 카드사 문의하시면 가능합니다.
2. 네, 가능합니다.
3. 네, 주문 시 결제 수단에서 소액 결제로 가능합니다.

✅ 맘에 드는 응답 번호를 선택하세요 (1~3): 2

🎯 최종 선택된 응답: 네, 가능합니다.

📥 질문을 입력하세요 (종료하려면 'exit'): 결제 취소해주세요

🔍 생성된 응답들:
1. 네, 주문되었습니다.
2. 네, 주문되었습니다.
3. 네, 결제 취소가 완료되었습니다.

✅ 맘에 드는 응답 번호를 선택하세요 (1~3): 3

🎯 최종 선택된 응답: 네, 결제 취소가 완료되었습니다.

📥 질문을 입력하세요 (종료하려면 'exit'): 일시불로 결제하면 할인 있나요?

🔍 생성된 응답들:
1. 네, 고객님 일시불 할인이 있습니다.
2. 네, 지금 방송 중인 상품 모두 십 프로 할인이 진행되고 있습니다.
3. 네, 카드 청구 시점에 따라 달라지니까 고객님께서 나중에 납부하실 때 적용 받으시면 십 퍼센트 추가 할인이 있습니다.

✅ 맘에 드는 응답 번호를 선택하세요 (1~3): exit
⚠️ 숫자로 입력해주세요.

✅ 맘에 드는 응답 번호를 선택하세요 (1~3): 1

🎯 최종 선택된 응답: 네, 고객님 일시불 할인이 있습니다.

📥 질문을 입력하세요 (종료하려면 'exit'): exit
